___
# Image Stacking and Classification
Author : _Sasika Amarasinghe_
___


In [ ]:
!gdown --id 14jMcdQRm5AlFYIwIWN_KH-kXyZIYeIhO

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=14jMcdQRm5AlFYIwIWN_KH-kXyZIYeIhO
From (redirected): https://drive.google.com/uc?id=14jMcdQRm5AlFYIwIWN_KH-kXyZIYeIhO&confirm=t&uuid=d8dcf863-6294-416d-af39-4722c9db631e
To: /content/training_data.rar
100% 1.37G/1.37G [00:31<00:00, 43.4MB/s]


In [ ]:
# prompt: unzip "/content/training_data.rar"

!unrar x "/content/training_data.rar"

Streaming output truncated to the last 5000 lines.
Extracting  ICIP training data/1/RawDataQA-1 (7)/RawDataQA-1-7 (95).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (7)/RawDataQA-1-7 (96).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (7)/RawDataQA-1-7 (97).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (7)/RawDataQA-1-7 (98).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (7)/RawDataQA-1-7 (99).tiff      72%  OK 
Creating    ICIP training data/1/RawDataQA-1 (8)                      OK
Extracting  ICIP training data/1/RawDataQA-1 (8)/RawDataQA-1-8 (1).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (8)/RawDataQA-1-8 (10).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (8)/RawDataQA-1-8 (100).tiff      72%  OK 
Extracting  ICIP training data/1/RawDataQA-1 (8)/RawDataQA-1-8 (101).tiff      72%  OK 
Extracting 

In [ ]:
from pathlib import Path
train_dir = Path("/content/ICIP training data/")

for dir in train_dir.iterdir():
  print(dir)

/content/ICIP training data/0
/content/ICIP training data/read me.txt
/content/ICIP training data/1
/content/ICIP training data/2


- 0: Normal dataset
- 1: Diabetes dataset
- 2: non-Diabetes dataset

In [ ]:
normal_dataset_dir = train_dir / "0"
diabetes_dataset_dir = train_dir / "1"
non_diabetes_dataset_dir = train_dir / "2"

In [ ]:
for i, dir in enumerate(normal_dataset_dir.iterdir()):
  if i == 5:
    break
  print(dir)

/content/ICIP training data/0/RawDataQA (5)
/content/ICIP training data/0/RawDataQA (33)
/content/ICIP training data/0/RawDataQA (25)
/content/ICIP training data/0/RawDataQA (41)
/content/ICIP training data/0/RawDataQA (3)


In [ ]:
print("Normal dataset :",len(list(normal_dataset_dir.iterdir())))
print("Diabetes dataset :", len(list(diabetes_dataset_dir.iterdir())))
print("Non diabetes dataset :",len(list(non_diabetes_dataset_dir.iterdir())))

Normal dataset : 42
Diabetes dataset : 30
Non diabetes dataset : 28


___
# TIFF Images Stacking
___


In [ ]:
list(normal_dataset_dir.iterdir())[0]

PosixPath('/content/ICIP training data/0/RawDataQA (5)')

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
import os

# Define the path to your original dataset directory and the new directory structure
new_base_dir = Path('/content/ICIP training data stacked')

# Ensure the new base directory exists
new_base_dir.mkdir(exist_ok=True)

# Define the file path structure
file_path_structure = {"ICIP training data stacked": [0, 1, 2]}

# Create subdirectories in the new base directory
for sub_dir_name in file_path_structure["ICIP training data stacked"]:
    new_sub_dir = new_base_dir / str(sub_dir_name)
    new_sub_dir.mkdir(exist_ok=True)


In [ ]:
new_normal_dataset_dir = Path("/content/ICIP training data stacked/0")
new_diabetes_dataset_dir = Path("/content/ICIP training data stacked/1")
new_non_diabetes_dataset_dir = Path("/content/ICIP training data stacked/2")

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np

def stack_tiff_images(dataset_dir, output_dir):
  # Iterate over all subdirectories
  for sub_dir in dataset_dir.iterdir():
      if sub_dir.is_dir():
          # List all .tiff files in the current subdirectory
          tiff_files = list(sub_dir.glob('*.tif*'))

          # Check if there are any .tiff files in the directory
          if tiff_files:
              # Load images and convert them to numpy arrays
              images = [np.array(Image.open(tiff_file)) for tiff_file in tiff_files]

              folder_name = sub_dir.name
              # print(folder_name)
              # print(len(images))
              len_images = len(images)
              # Stack the images along the first axis (depth-wise stacking)
              stacked_image = np.stack(images, axis=0)
              # print(stacked_image.shape)

              # Sum all the layers
              sum_image = np.sum(stacked_image, axis=0)
              # print(sum_image.shape)

              # Compute the average
              average_image = sum_image / len_images

              # Add a new axis to match the shape (1, 300, 300)
              average_image = np.expand_dims(average_image, axis=0)
              # print(average_image.shape)

              # Remove the extra dimension to get shape (300, 300)
              average_image = np.squeeze(average_image)
              # print(average_image.shape)

              # Convert the average numpy array back to an image
              average_image_pil = Image.fromarray(average_image.astype(np.uint8))

              output_path = output_dir

              # output_path =
              # # Save the average image
              output_path = output_path / f'{folder_name}.tiff'
              average_image_pil.save(output_path)

stack_tiff_images(normal_dataset_dir, new_normal_dataset_dir)
stack_tiff_images(diabetes_dataset_dir, new_diabetes_dataset_dir)
stack_tiff_images(non_diabetes_dataset_dir, new_non_diabetes_dataset_dir)

___
# Classification
___

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from PIL import Image

data_dir = '/content/ICIP training data stacked'

# Custom transform for TIFF images
class TIFFImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

# Define your custom transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match VGG input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset using the custom TIFFImageFolder
dataset = TIFFImageFolder(root=data_dir, transform=transform)

# Load all the data at once
data_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=True)

# Unpack the data loader into inputs and labels
inputs, labels = next(iter(data_loader))

# Check the shape of inputs and labels
print("Inputs shape:", inputs.shape)
print("Labels shape:", labels.shape)


Inputs shape: torch.Size([100, 3, 224, 224])
Labels shape: torch.Size([100])


In [ ]:
import torch.nn as nn
import torchvision.models as models

num_classes = 3  # Change this to the number of classes you have

model = models.vgg16(pretrained=True)
# Freeze all the parameters
for param in model.parameters():
    param.requires_grad = False

# Modify the classifier
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier[6].parameters(), lr=0.001)

In [ ]:
train_loader = data_loader

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # # Validation phase
    # model.eval()
    # val_loss = 0.0
    # corrects = 0
    # with torch.no_grad():
    #     for inputs, labels in val_loader:
    #         inputs, labels = inputs.to(device), labels.to(device)
    #         outputs = model(inputs)
    #         loss = criterion(outputs, labels)
    #         val_loss += loss.item() * inputs.size(0)
    #         _, preds = torch.max(outputs, 1)
    #         corrects += torch.sum(preds == labels.data)

    # val_loss = val_loss / len(val_loader.dataset)
    # val_acc = corrects.double() / len(val_loader.dataset)
    # print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')


Epoch 1/100, Loss: 0.3106
Epoch 2/100, Loss: 0.3277
Epoch 3/100, Loss: 0.3421
Epoch 4/100, Loss: 0.3438
Epoch 5/100, Loss: 0.3569
Epoch 6/100, Loss: 0.3861
Epoch 7/100, Loss: 0.3731
Epoch 8/100, Loss: 0.3494
Epoch 9/100, Loss: 0.2844
Epoch 10/100, Loss: 0.2741
Epoch 11/100, Loss: 0.3505
Epoch 12/100, Loss: 0.3464
Epoch 13/100, Loss: 0.4217
Epoch 14/100, Loss: 0.3647
Epoch 15/100, Loss: 0.3394
Epoch 16/100, Loss: 0.3404
Epoch 17/100, Loss: 0.3436
Epoch 18/100, Loss: 0.3556
Epoch 19/100, Loss: 0.3542
Epoch 20/100, Loss: 0.3197
Epoch 21/100, Loss: 0.3129
Epoch 22/100, Loss: 0.2830
Epoch 23/100, Loss: 0.3116
Epoch 24/100, Loss: 0.3950
Epoch 25/100, Loss: 0.3303
Epoch 26/100, Loss: 0.2737
Epoch 27/100, Loss: 0.2770
Epoch 28/100, Loss: 0.3041
Epoch 29/100, Loss: 0.3165
Epoch 30/100, Loss: 0.3647
Epoch 31/100, Loss: 0.3149
Epoch 32/100, Loss: 0.3314
Epoch 33/100, Loss: 0.3746
Epoch 34/100, Loss: 0.3213
Epoch 35/100, Loss: 0.2762
Epoch 36/100, Loss: 0.3437
Epoch 37/100, Loss: 0.3405
Epoch 38/1

In [ ]:
torch.save(model.state_dict(), f'vgg_transfer_learning_{num_epochs}_epochs.pth')

___
# Validation
___

In [ ]:
# prompt: Download this and unrar "https://drive.google.com/file/d/1pedaOofYKEC4eTjiniD3jEyN1w3K7jDu/view?usp=sharing"

!gdown --id 1pedaOofYKEC4eTjiniD3jEyN1w3K7jDu

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1pedaOofYKEC4eTjiniD3jEyN1w3K7jDu
From (redirected): https://drive.google.com/uc?id=1pedaOofYKEC4eTjiniD3jEyN1w3K7jDu&confirm=t&uuid=bfac7bcf-b37c-4afc-8f10-7130a1671e51
To: /content/ICIP_test_data_final.rar
100% 162M/162M [00:01<00:00, 98.6MB/s]


In [ ]:
!unrar x /content/ICIP_test_data_final.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/ICIP_test_data_final.rar

Creating    ICIP_test_data_final                                      OK
Creating    ICIP_test_data_final/RawDataQA (1)                        OK
Extracting  ICIP_test_data_final/RawDataQA (1)/(1).tif                     0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(10).tif                    0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(100).tif                   0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(101).tif                   0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(102).tif                   0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(103).tif                   0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(104).tif                   0%  OK 
Extracting  ICIP_test_data_final/RawDataQA (1)/(105).tif                   0%

In [ ]:
test_data_dir = Path("/content/ICIP_test_data_final")

In [ ]:
os.makedirs("/content/test_data_stacked")
stacked_test_data_dir = Path("/content/test_data_stacked")

In [ ]:
stack_tiff_images(test_data_dir, stacked_test_data_dir )

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision import models
import numpy as np
from PIL import Image

# Load the pretrained VGG model
model = models.vgg16(pretrained=True)
# Modify the classifier
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, num_classes)

# Load the model weights
model.load_state_dict(torch.load("/content/vgg_transfer_learning_100_epochs.pth"))
model.eval();

In [ ]:
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms

# Load the pretrained VGG model
model = models.vgg16(pretrained=True)

# Modify the classifier
num_classes = 3  # Assuming your model was trained on 1000 classes
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, num_classes)

# Load the model weights
model.load_state_dict(torch.load("/content/vgg_transfer_learning_100_epochs.pth"))
model.eval()

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match VGG input size
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Initialize an empty list to store predictions
all_predictions = dict()

# Iterate over the images
for image_path in stacked_test_data_dir.glob('*.tif*'):
    # Load the image
    image = Image.open(str(image_path))
    image_name = str(image_path).split("/")[-1]

    # Apply the transformation
    input_image = transform(image).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        outputs = model(input_image)

    # Get the predicted class
    _, predicted = torch.max(outputs, 1)

    # Append the predicted class to the list
    all_predictions[image_name] = predicted.item()

# Convert the list of predictions to a numpy array
# all_predictions = np.array(all_predictions)

# Print the predictions
print("Predictions:", all_predictions)

Predictions: {'RawDataQA (2).tiff': 2, 'RawDataQA (9).tiff': 0, 'RawDataQA (11).tiff': 1, 'RawDataQA (5).tiff': 1, 'RawDataQA (17).tiff': 2, 'RawDataQA (10).tiff': 0, 'RawDataQA (14).tiff': 1, 'RawDataQA (1).tiff': 2, 'RawDataQA (12).tiff': 1, 'RawDataQA (4).tiff': 1, 'RawDataQA (15).tiff': 2, 'RawDataQA (7).tiff': 1, 'RawDataQA (13).tiff': 2, 'RawDataQA (16).tiff': 1, 'RawDataQA (3).tiff': 0, 'RawDataQA (18).tiff': 0, 'RawDataQA (8).tiff': 1, 'RawDataQA (6).tiff': 1}


In [ ]:
for key in all_predictions.keys():
  print(key, ":", all_predictions[key])

RawDataQA (2).tiff : 2
RawDataQA (9).tiff : 0
RawDataQA (11).tiff : 1
RawDataQA (5).tiff : 1
RawDataQA (17).tiff : 2
RawDataQA (10).tiff : 0
RawDataQA (14).tiff : 1
RawDataQA (1).tiff : 2
RawDataQA (12).tiff : 1
RawDataQA (4).tiff : 1
RawDataQA (15).tiff : 2
RawDataQA (7).tiff : 1
RawDataQA (13).tiff : 2
RawDataQA (16).tiff : 1
RawDataQA (3).tiff : 0
RawDataQA (18).tiff : 0
RawDataQA (8).tiff : 1
RawDataQA (6).tiff : 1


In [ ]:
RawDataQA (2).tiff : 2
RawDataQA (9).tiff : 0
RawDataQA (11).tiff : 1
RawDataQA (5).tiff : 1
RawDataQA (17).tiff : 2
RawDataQA (10).tiff : 0
RawDataQA (14).tiff : 1
RawDataQA (1).tiff : 2
RawDataQA (12).tiff : 1
RawDataQA (4).tiff : 1
RawDataQA (15).tiff : 2
RawDataQA (7).tiff : 1
RawDataQA (13).tiff : 2
RawDataQA (16).tiff : 1
RawDataQA (3).tiff : 0
RawDataQA (18).tiff : 0
RawDataQA (8).tiff : 1
RawDataQA (6).tiff : 1